#  Setup: Display options

In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


# Setup: Connectivity

In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


# Step 01:  Process CSV files ..

In [ ]:

#  This data was sourced from,
#     https://mtsamples.com/site/pages/browse.asp?type=96-Hematology%20-%20Oncology
#

#  Column headers,
#     id|medical_specialty|sample_name|description|body
#
df_data      = pd.read_csv("./10_Data/22_OncologyCallNotes.txt", header = "infer", delimiter = "|")


#  90 Real lines of data.  (Some data is multi-line, and enclosed in double quotes.)
#
print("Number of CSV input lines: %d" % (len(df_data)))
print("")


#   Above, the field titled "body" contains embedded new lines; remove those
#   and place in a new field titled, "body_1line"
#
df_data["body_1line"] = df_data.body.map(lambda x: str(x).replace("\n", "") )


l_cntr = 0
   #
for l_each in df_data.itertuples():
   l_cntr += 1
      #
   if (l_cntr < 5):
      print("Record number: %d   Sample name: %-44s   Short text: %s" % (l_each.id, l_each.sample_name[0:43], l_each.body_1line[0:60]))
        
        
print("")
print("--")

#  Sample output,
#
#     Number of CSV input lines: 90
#     
#     Record number: 4001   Sample name: 3-Dimensional Simulation                       Short text: This patient is undergoing 3-dimensionally planned radiation
#     Record number: 4002   Sample name: Adrenalectomy & Umbilical Hernia Repair        Short text: PREOPERATIVE DIAGNOSES1. Adrenal mass, right sided.2. Umbi
#     Record number: 4003   Sample name: Anaplastic Astrocytoma - Letter                Short text: Month DD, YYYYXYZRE: ABCMEDICAL RECORD#: 123Dear Dr. 
#     Record number: 4004   Sample name: Anemia & Leukemia Followup                     Short text: CHIEF COMPLAINT:1. Chronic lymphocytic leukemia (CLL).2. A


#  Step 02: Enrich the above from a Google Web service ..

In [ ]:

#  Google has a Web service to convert text into usable UMLS codes. See,
#        https://cloud.google.com/healthcare-api/docs/how-tos/nlp
#
#  In this cell, we begin to invoke this service on the text from the
#  cell above.

#  See also,
#     https://stackoverflow.com/questions/53472429/how-to-get-a-gcp-bearer-token-programmatically-with-python

#  Google:
#
#     .  We had to create an Auth Token, which produced a JSON file.
#        (Instruction in Url above.)
#
#     .  Our JSON file is at,
#              export GOOGLE_APPLICATION_CREDENTIALS="/mnt/hgfs/My.20/MyShare_1/46 Topics 2022/91 KG, All Prospects/13 KG, DataBricks, Google/10_Data/05_katana-clusters-beta-d8605ac248e7.json"
#              export GOOGLE_APPLICATION_CREDENTIALS="/home/jovyan/work/My_KG_NoteBooks/P1_Prospects/10_DataBricks_Google/10_Data/05_katana-clusters-beta-d8605ac248e7.json"
#
#     .  To extract the Auth Token, set the above, then run
#           gcloud auth application-default print-access-token


import google.auth
import google.auth.transport.requests
from google.oauth2 import service_account


#  This token times out often; you must rerun this block from time to time
#
l_credentials = service_account.Credentials.from_service_account_file(
   "/home/jovyan/work/My_KG_NoteBooks/P1_Prospects/10_DataBricks_Google/10_Data/05_katana-clusters-beta-d8605ac248e7.json",
   scopes=['https://www.googleapis.com/auth/cloud-platform'])
l_auth_req = google.auth.transport.requests.Request()
l_credentials.refresh(l_auth_req)
   #
l_token = l_credentials.token
    
print(l_token)


#  Sample output,
#
#     ya29.c.b0Aa9VdylvtWgGXBZyFkW5mADfcFiyBVitZsvkoKbHpCuXU7zGkgANRcho_ax5_SWWbiXfQj6cprlobWUlHnPkYEoKBCRw6   ...
    

In [ ]:

#  Cool page, convert Curl command to Python,
#     https://reqbin.com/curl

#  Run the Google Web service, capture results
#

import requests
from requests.structures import CaseInsensitiveDict
   #
import json

    
#  Function, because we will loop on this below-
#
def f_enrich(i_arg1):
    
   url = "https://healthcare.googleapis.com/v1/projects/katana-clusters-beta/locations/us-central1/services/nlp:analyzeEntities"
   
   l_headers = CaseInsensitiveDict()
      #
   l_headers["Authorization"] = "Bearer " + l_token
   l_headers["Content-Type"]  = "application/json"
       
    
   l_data = """
      {{
      'nlpService':'projects/katana-clusters-beta/locations/us-central1/services/nlp',
      'documentContent':'{0}'
      }}
      """.format(i_arg1)
         #
   l_resp = requests.post(url, headers = l_headers, data = l_data)
      #
   return l_resp



l_response = f_enrich("Insulin regimen human 5 units IV administered.")
   #

print(l_response.status_code)
print("")
   #
l_data_asjson = json.loads(l_response.content)                       #  Get the response in json
print(json.dumps(l_data_asjson, indent = 3))                         #  This gives us a pretty print (easier to read)


#  Sample data after this cell-,
#


#  Step 00:  Sample data from above

In [ ]:

#  Sample data from above,
#
#     200
#     
#     {
#        "entityMentions": [
#           {
#              "mentionId": "1",
#              "type": "MEDICINE",
#              "text": {
#                 "content": "Insulin regimen",
#                 "beginOffset": 0
#              },
#              "linkedEntities": [
#                 {
#                    "entityId": "UMLS/C0021641"
#                 },
#                 {
#                    "entityId": "UMLS/C0795635"
#                 },
#                 {
#                    "entityId": "UMLS/C1533581"
#                 },
#                 {
#                    "entityId": "UMLS/C3537244"
#                 },
#                 {
#                    "entityId": "UMLS/C3714501"
#                 }
#              ],
#              "temporalAssessment": {
#                 "value": "CURRENT",
#                 "confidence": 0.8573660850524902
#              },
#              "certaintyAssessment": {
#                 "value": "LIKELY",
#                 "confidence": 0.9751282930374146
#              },
#              "subject": {
#                 "value": "PATIENT",
#                 "confidence": 0.9995787739753723
#              },
#              "confidence": 0.6379408836364746
#           },
#           {
#              "mentionId": "2",
#              "type": "MED_DOSE",
#              "text": {
#                 "content": "5 units",
#                 "beginOffset": 22
#              },
#              "confidence": 0.7443782091140747
#           },
#           {
#              "mentionId": "3",
#              "type": "MED_ROUTE",
#              "text": {
#                 "content": "IV",
#                 "beginOffset": 30
#              },
#              "linkedEntities": [
#                 {
#                    "entityId": "UMLS/C0348016"
#                 }
#              ],
#              "confidence": 0.779011607170105
#           }
#        ],
#        "entities": [
#           {
#              "entityId": "UMLS/C0021641",
#              "preferredTerm": "Insulin",
#              "vocabularyCodes": [
#                 "FMA/83365",
#                 "LNC/LA15805-7",
#                 "LNC/LP14676-8",
#                 "LNC/LP16325-0",
#                 "LNC/LP32542-0",
#                 "LNC/LP70329-5",
#                 "LNC/MTHU002108",
#                 "LNC/MTHU019392",
#                 "MSH/D007328",
#                 "MTH/NOCODE"
#              ]
#           },
#           {
#              "entityId": "UMLS/C0348016",
#              "preferredTerm": "Intravenous",
#              "vocabularyCodes": [
#                 "LNC/LA9437-0",
#                 "LNC/LP32453-0",
#                 "MTH/NOCODE",
#                 "NCI/C13346"
#              ]
#           },
#           {
#              "entityId": "UMLS/C0795635",
#              "preferredTerm": "insulin, regular, human",
#              "vocabularyCodes": [
#                 "LNC/LP17001-6",
#                 "MSH/D061386",
#                 "MTH/NOCODE",
#                 "NCI/C29125",
#                 "RXNORM/253182",
#                 "VANDF/4017559",
#                 "VANDF/4017569",
#                 "VANDF/4019786"
#              ]
#           },
#           {
#              "entityId": "UMLS/C1533581",
#              "preferredTerm": "Therapeutic Insulin",
#              "vocabularyCodes": [
#                 "MTH/NOCODE",
#                 "NCI/C581"
#              ]
#           },
#           {
#              "entityId": "UMLS/C3537244",
#              "preferredTerm": "Insulins",
#              "vocabularyCodes": [
#                 "MSH/D061385",
#                 "MTH/NOCODE"
#              ]
#           },
#           {
#              "entityId": "UMLS/C3714501",
#              "preferredTerm": "Insulin Drug Class",
#              "vocabularyCodes": [
#                 "MTH/NOCODE",
#                 "VANDF/4021631"
#              ]
#           }
#        ],
#        "relationships": [
#           {
#              "subjectId": "1",
#              "objectId": "2",
#              "confidence": 0.9996469616889954
#           },
#           {
#              "subjectId": "1",
#              "objectId": "3",
#              "confidence": 0.9995671510696411
#           }
#        ]
#     }


#  Step 02:  (continued)

In [ ]:

#  Run the above Web service against our first DataFrame
#

#  Recall df_data contains,
#
#     id|medical_specialty|sample_name|description|body|body_1line
#


df_enriched = pd.DataFrame(columns = ["id", "enrich_from_Google"])


for l_each in df_data.itertuples():
   print("Processing id: %d" % (l_each.id))
      #
   l_response = f_enrich(l_each.body_1line)
   l_data_asjson = json.loads(l_response.content) 

   df_enriched = df_enriched.append({"id" : l_each.id, "enrich_from_Google" : l_data_asjson}, ignore_index = True)
    

print("")


l_cntr = 0
   #
for l_each in df_enriched.itertuples():
   l_cntr += 1
      #
   if (l_cntr < 5):
      print("Record number: %d   Short text: %s" % (l_each.id, str(l_each.enrich_from_Google)[0:60]))
        
        
print("")
print("--")

#  Sample output,
#
#     Processing id: 4001
#     Processing id: 4002
#     Processing id: 4003
#        ...
#         
#     Record number: 4001   Short text: {'error': {'code': 400, 'message': "Invalid JSON payload rec
#     Record number: 4002   Short text: {'entityMentions': [{'mentionId': '1', 'type': 'PROBLEM', 't
#     Record number: 4003   Short text: {'entityMentions': [{'mentionId': '1', 'type': 'PROBLEM', 't
#     Record number: 4004   Short text: {'entityMentions': [{'mentionId': '1', 'type': 'PROBLEM', 't
#     
#     --


In [ ]:

#  Write this out to a file
#

l_file = "./10_Data/23_22_PlusGoogle.txt"

df_enriched.to_csv(l_file, sep = "|", encoding = "utf-8", index = False)


print("--")


# Step 03:  Extract actual field data from the JSON Google gave us

In [53]:


#  The model for this DataFrame is,
#     id|enrich_from_Google
#
#     with the second column being a JSON encoded string with its own model.
#

l_NumOfBadJson    = 0
l_RootKeyNotFound = 0
   #
l_EntityIds       = set()
l_VocabCodes      = set()


for l_each in df_enriched.itertuples():
    
   #  Before we cleaned up all of the JSON (random single quotes for possesive nouns,
   #  other), we would get errors similar to,
   #
   #      4001|"{'error': {'code': 400, 'message': ""Invalid JSON payload received. Expected , or } after key:value pair.\naced
   #         on the patient's skin or on the imm\n                    ^"", 'status': 'INVALID_ARGUMENT'}}"
   #
   #  Upon receipt of this condition, a root level key with the title "error" would be
   #  present. If those return, filter those out-
   #
   if ("error" in l_each.enrich_from_Google):
      l_NumOfBadJson += 1
   else:
      if ("entities" in l_each.enrich_from_Google):
            
         for l_entity in l_each.enrich_from_Google["entities"]:
            print(l_entity["entityId"])
            
            
            
            
            
            
            
      else:
         l_RootKeyNotFound += 1
    
    
    
    
    
    
print("--")    
    
    
    
    
    

UMLS/C0009244
UMLS/C0027651
UMLS/C0034606
UMLS/C0040405
UMLS/C0086066
UMLS/C0183115
UMLS/C0184661
UMLS/C0205143
UMLS/C0441633
UMLS/C0678594
UMLS/C0879626
UMLS/C0936234
UMLS/C1123023
UMLS/C1306645
UMLS/C1522449
UMLS/C1608329
UMLS/C1859506
UMLS/C1859507
UMLS/C1962945
UMLS/C3263145
UMLS/C3273245
UMLS/C3273930
UMLS/C4289670
UMLS/C4520765
UMLS/C4721829
UMLS/C0001625
UMLS/C0002903
UMLS/C0005841
UMLS/C0005847
UMLS/C0009368
UMLS/C0009450
UMLS/C0011377
UMLS/C0012737
UMLS/C0015641
UMLS/C0019080
UMLS/C0019116
UMLS/C0019333
UMLS/C0022646
UMLS/C0023884
UMLS/C0027651
UMLS/C0031150
UMLS/C0035092
UMLS/C0038928
UMLS/C0041158
UMLS/C0041638
UMLS/C0042460
UMLS/C0087111
UMLS/C0156119
UMLS/C0175722
UMLS/C0179804
UMLS/C0183014
UMLS/C0183177
UMLS/C0183362
UMLS/C0183461
UMLS/C0184661
UMLS/C0184898
UMLS/C0185115
UMLS/C0221198
UMLS/C0226407
UMLS/C0226717
UMLS/C0227425
UMLS/C0230195
UMLS/C0232808
UMLS/C0342500
UMLS/C0392220
UMLS/C0439663
UMLS/C0441126
UMLS/C0444912
UMLS/C0446443
UMLS/C0553891
UMLS/C0582240
UMLS/C

In [ ]:

#  In a Web browser
#     
#     https://console.cloud.google.com/storage/browser/a9187b90-cluster/kind-mullet/Paypal_20M_1B/
#

from google.cloud import storage


l_client      = storage.Client()

l_bucket_str  = "a9187b90-cluster"
   #
l_bucket      = l_client.bucket(l_bucket_str)


#  Just nodes
#
l_nodes = []
   #
for l_each in l_bucket.list_blobs(prefix = "kind-mullet/Paypal_20M_1B/N"):
   print(l_each.name)
      #
   l_nodes.append("gs://" + l_bucket_str + "/" + l_each.name)

print(l_nodes)


print("--")


#  Just edges
#
l_edges       = []
   #
l_cntr        = 0                                   #  We'll use these next two lines for testing
l_edges_small = []
   #
for l_each in l_bucket.list_blobs(prefix = "kind-mullet/Paypal_20M_1B/f"):
   print(l_each.name)
      #
   l_edges.append("gs://" + l_bucket_str + "/" + l_each.name)
      #
   l_cntr += 1
   if (l_cntr < 3):
      l_edges_small.append("gs://" + l_bucket_str + "/" + l_each.name)

print(l_edges)
print(l_edges_small)


print("--")

#  Sample output,
#
#     kind-mullet/Paypal_20M_1B/NodeFile_0
#     ['gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/NodeFile_0']
#     --
#     kind-mullet/Paypal_20M_1B/fileAssetA_0
#     kind-mullet/Paypal_20M_1B/fileAssetA_1
#     kind-mullet/Paypal_20M_1B/fileAssetA_2
#     kind-mullet/Paypal_20M_1B/fileAssetA_3
#     kind-mullet/Paypal_20M_1B/fileAssetA_4
#     kind-mullet/Paypal_20M_1B/fileAssetA_5
#     kind-mullet/Paypal_20M_1B/fileAssetB_0
#     kind-mullet/Paypal_20M_1B/fileAssetB_1
#     kind-mullet/Paypal_20M_1B/fileAssetB_2
#     kind-mullet/Paypal_20M_1B/fileAssetB_3
#     kind-mullet/Paypal_20M_1B/fileFriendA_0
#     kind-mullet/Paypal_20M_1B/fileFriendB_0
#     kind-mullet/Paypal_20M_1B/fileFriendB_1
#     ['gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/fileAssetA_0', 'gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/fileAssetA_1', 'gs://a9187b90-cluster/kind-mullet/P  ...
#     ['gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/fileAssetA_0', 'gs://a9187b90-cluster/kind-mullet/Paypal_20M_1B/fileAssetA_1']
#     --


#  Step 02: Read CSVs into Dask DataFrame

In [ ]:

import dask.dataframe as dd
import numpy as np

print("--")


In [ ]:
%%time

#  Just nodes

#  Sample data,
#
#     Sent|counterparty|cntAppr|cntDecl|cntRiskDecl1|amtRiskDecl|cntRiskDecl2|sumRcvd|cntRcvd|cntDist|cntDecl2
#     0|N|N|N|N|XbRqfTcGJL9fxmNp4simTykwKr|qhBymw13PneK5hpntNVwjNwaYklX8P379JZo|Y|416|0|inactive|1941730454454|0|2018-01-24T01:09:18|9.07632e+09|1419493889|1894897856|2676929303.1430|6614868331.8687|8283173087.0037|6020979256.1980|1984359695|5077917542.6596|1923699935|1948138778|4876422147.1566|

df_nodes = dd.read_csv(
   l_nodes,
   delimiter = "|",
   dtype = {
      "Sent"          : np.dtype("O"),                         #  Change these if you wish,  Numpy datatypes
      "counterparty"  : np.dtype("O"),
      "cntAppr"       : np.dtype("O"),
      "cntDecl"       : np.dtype("O"),
      "cntRiskDecl1"  : np.dtype("O"),
      "amtRiskDecl"   : np.dtype("O"),
      "cntRiskDecl2"  : np.dtype("O"),
      "sumRcvd"       : np.dtype("O"),
      "cntRcvd"       : np.dtype("O"),
      "cntDist"       : np.dtype("O"),
      "cntDecl2"      : np.dtype("O"),
      })

print(len(df_nodes))

print("--")

#  Sample output,
#
#     20000000
#     --
#     CPU times: user 3min, sys: 18 s, total: 3min 18s
#     Wall time: 2min 48s


In [ ]:
%%time

#  Just edges

#  Sample data,
#
#     Account1.id|Account2.id|type|date
#     16089173|16089175|FriendConB|2018-10-10T11:20:05

df_edges = dd.read_csv(
    
   #  l_edges,                                                   #  This would be all files, at once, in memory
   l_edges_small,
    
   delimiter = "|",
   dtype = {
      "Account1.id"   : np.dtype("O"),
      "Account2.id"   : np.dtype("O"),
      "type"          : np.dtype("O"),                           #  These next two column names are keywords in most worlds -- bad idea
      "date"          : np.dtype("O"),
      })

print(len(df_edges))

print("--")

#  Sample output,
#
#     200000000
#     --
#     CPU times: user 4min 25s, sys: 31.2 s, total: 4min 56s
#     Wall time: 4min 23s
